In [1]:
import torch
import faiss
import numpy as np
import torchvision
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.decomposition import PCA
from torchvision import transforms as T
from torchvision.models.vgg import vgg19
from torchvision.models.resnet import resnet50


img = Image.open('../data/test-elephant.png')
print(img)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1062 at 0x1289C87D0>


## Load models

In [2]:
vgg_model = vgg19()
resnet_model = resnet50()

## Test DinoV2

In [3]:
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')

transform = T.Compose([
    T.Resize(520),
    T.CenterCrop(518),           
    T.ToTensor(),               
    T.Normalize(mean=0.5, std=0.2)
])

patch_size = dinov2.patch_size # patchsize=14

patch_h  = 520//patch_size
patch_w  = 520//patch_size

feat_dim = 1024

img = transform(img)

with torch.no_grad():
    features_dict = dinov2.forward_features(img.unsqueeze(0))
    features = features_dict['x_norm_patchtokens']

# pca = PCA(n_components=3)
# pca.fit(features)

# pca_features = pca.transform(features)
# pca_features = (pca_features - pca_features.min()) / (pca_features.max() - pca_features.min())
# pca_features = pca_features * 255

# plt.imshow(pca_features.reshape(16, 16, 3).astype(np.uint8))
# plt.savefig('meta_dog_features.png')

# In dinov2/models/vision_transformer.py line 290 add

# def forward(self, *args, is_training=False, return_patches=False, **kwargs):
# ret = self.forward_features(*args, **kwargs)
# if is_training:
# return ret
# elif return_patches:
# return ret["x_norm_patchtokens"]
# else:
# return self.head(ret["x_norm_clstoken"])

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /Users/michal/.cache/torch/hub/main.zip
/Users/michal/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/michal/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/michal/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /Users/michal/.cache/torch/hub/checkpoints/dinov2_vitl14_pretrain.pth
100%|██████████| 1.13G/1.13G [02:18<00:00, 8.82MB/s]


## Test FAISS